In [1]:
import pandas as pd
import janitor

In [2]:
# Group and keep columns with 'as_index'
# https://stackoverflow.com/questions/31569549/how-to-groupby-a-dataframe-in-pandas-and-keep-columns

### Daily report data

In [3]:
data = pd.read_csv('../latest_raw.csv', encoding='utf-8')
data = data.drop(['MUNICIPIO_RES'], axis=1) # readily drop this column

In [4]:
print(data.set_index(['ID_REGISTRO']).index.is_unique)

False


In [5]:
data['FECHA_INGRESO'] = pd.to_datetime(data.FECHA_INGRESO, format='%Y-%m-%d', errors='coerce')
data['FECHA_INGRESO'] = data['FECHA_INGRESO'].dt.strftime('%Y-%m-%d')
data = data.set_index(pd.DatetimeIndex(data['FECHA_INGRESO'])).sort_index()

In [6]:
len(data)

1571956

### Note
Daily report for april 20 (ie, reported in 21 april) stops using testing site (ENTIDAD_UM),
<br>
And starts using region of residence (ENTIDAD_RES) so we shall treat both sets of data accordingly

In [7]:
dat = data.copy()

In [8]:
tbl_x = dat.filter_date('FECHA_INGRESO','2020-01-06','2020-04-20')
tbl_x = tbl_x.drop(['ENTIDAD_RES'], axis=1) # drop by region
tbl_x = tbl_x.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO']
)
tbl_x = tbl_x.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_UM', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO'], as_index=True).count().reset_index()
tbl_x = tbl_x.rename(columns={'ENTIDAD_UM':'Region_ID'})

In [9]:
tbl_y = dat.filter_date('FECHA_INGRESO','2020-04-21','2022-12-31') # the long con
tbl_y = tbl_y.drop(['ENTIDAD_UM'], axis=1) # drop by region
tbl_y = tbl_y.set_index(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO']
)
tbl_y = tbl_y.groupby(
    ['FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'ENTIDAD_RES', 'ID_REGISTRO', 'PAIS_ORIGEN', 'RESULTADO'], as_index=True).count().reset_index()
tbl_y = tbl_y.rename(columns={'ENTIDAD_RES':'Region_ID'})

In [10]:
tbl = pd.concat([tbl_x, tbl_y])

In [11]:
tbl

,FECHA_INGRESO,FECHA_SINTOMAS,FECHA_DEF,Region_ID,ID_REGISTRO,PAIS_ORIGEN,RESULTADO
0,2020-01-06,2020-01-02,9999-99-99,14,05afcc,Local,2
1,2020-01-06,2020-01-02,9999-99-99,23,1c405f,Local,2
2,2020-01-06,2020-01-03,9999-99-99,5,0131f0,Local,2
3,2020-01-06,2020-01-03,9999-99-99,9,105f1c,Local,2
4,2020-01-06,2020-01-03,9999-99-99,14,01fefc,Local,2
...,...,...,...,...,...,...,...
82009,2020-05-05,2020-05-05,9999-99-99,28,1cc00f,Local,3
82010,2020-05-05,2020-05-05,9999-99-99,29,025692,Local,3
82011,2020-05-05,2020-05-05,9999-99-99,29,04f4ab,Local,3
82012,2020-05-05,2020-05-05,9999-99-99,30,1aacf9,Local,3


In [12]:
tbl = tbl.rename(
    columns={
        'FECHA_INGRESO':'Date_Confirmed',
        'FECHA_SINTOMAS':'Date_Symptoms',
        'FECHA_DEF':'Date_Death',
        'ID_REGISTRO':'Case_ID',
        'RESULTADO':'Status',
        'PAIS_ORIGEN':'Origin'
    })

In [13]:
tbl

,Date_Confirmed,Date_Symptoms,Date_Death,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,14,05afcc,Local,2
1,2020-01-06,2020-01-02,9999-99-99,23,1c405f,Local,2
2,2020-01-06,2020-01-03,9999-99-99,5,0131f0,Local,2
3,2020-01-06,2020-01-03,9999-99-99,9,105f1c,Local,2
4,2020-01-06,2020-01-03,9999-99-99,14,01fefc,Local,2
...,...,...,...,...,...,...,...
82009,2020-05-05,2020-05-05,9999-99-99,28,1cc00f,Local,3
82010,2020-05-05,2020-05-05,9999-99-99,29,025692,Local,3
82011,2020-05-05,2020-05-05,9999-99-99,29,04f4ab,Local,3
82012,2020-05-05,2020-05-05,9999-99-99,30,1aacf9,Local,3


In [14]:
tbl

,Date_Confirmed,Date_Symptoms,Date_Death,Region_ID,Case_ID,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,14,05afcc,Local,2
1,2020-01-06,2020-01-02,9999-99-99,23,1c405f,Local,2
2,2020-01-06,2020-01-03,9999-99-99,5,0131f0,Local,2
3,2020-01-06,2020-01-03,9999-99-99,9,105f1c,Local,2
4,2020-01-06,2020-01-03,9999-99-99,14,01fefc,Local,2
...,...,...,...,...,...,...,...
82009,2020-05-05,2020-05-05,9999-99-99,28,1cc00f,Local,3
82010,2020-05-05,2020-05-05,9999-99-99,29,025692,Local,3
82011,2020-05-05,2020-05-05,9999-99-99,29,04f4ab,Local,3
82012,2020-05-05,2020-05-05,9999-99-99,30,1aacf9,Local,3


In [15]:
geo = pd.read_csv('../data/geo/entidades.csv')

In [16]:
geo.keys()

Index(['CLAVE_ENTIDAD', 'ENTIDAD_FEDERATIVA', 'ABREVIATURA'], dtype='object')

In [17]:
geo = geo.rename(columns={'CLAVE_ENTIDAD':'Region_ID', 'ENTIDAD_FEDERATIVA':'Region', 'ABREVIATURA':'Region_Key'})

In [18]:
output = pd.merge(tbl, geo, on='Region_ID') 

In [19]:
output = output[['Date_Confirmed', 'Date_Symptoms', 'Date_Death', 'Region', 'Region_ID', 'Region_Key', 'Origin', 'Status']]

In [20]:
output

,Date_Confirmed,Date_Symptoms,Date_Death,Region,Region_ID,Region_Key,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
1,2020-01-06,2020-01-03,9999-99-99,JALISCO,14,JC,Local,2
2,2020-01-06,2020-01-06,9999-99-99,JALISCO,14,JC,Local,2
3,2020-01-07,2020-01-01,9999-99-99,JALISCO,14,JC,Local,2
4,2020-01-07,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
...,...,...,...,...,...,...,...,...
171606,2020-05-04,2020-04-30,9999-99-99,COLIMA,6,CM,Local,3
171607,2020-05-04,2020-04-30,9999-99-99,COLIMA,6,CM,Local,3
171608,2020-05-04,2020-05-02,9999-99-99,COLIMA,6,CM,Local,3
171609,2020-05-04,2020-05-03,9999-99-99,COLIMA,6,CM,Local,3


In [21]:
output['Region'] = output['Region'].str.replace('COAHUILA DE ZARAGOZA','COAHUILA')
output['Region'] = output['Region'].str.replace('MICHOACÁN DE OCAMPO','MICHOACAN')
output['Region'] = output['Region'].str.replace('VERACRUZ DE IGNACIO DE LA LLAVE','VERACRUZ')

In [22]:
output = output.sort_values(['Date_Confirmed', 'Date_Symptoms', 'Region'])

In [23]:
output.Region.unique()

array(['JALISCO', 'QUINTANA ROO', 'CIUDAD DE MÉXICO', 'COAHUILA',
       'VERACRUZ', 'BAJA CALIFORNIA', 'DURANGO', 'TABASCO', 'MÉXICO',
       'NAYARIT', 'SONORA', 'GUANAJUATO', 'MORELOS', 'TAMAULIPAS',
       'HIDALGO', 'PUEBLA', 'AGUASCALIENTES', 'BAJA CALIFORNIA SUR',
       'YUCATÁN', 'MICHOACAN', 'GUERRERO', 'OAXACA', 'SINALOA',
       'NUEVO LEÓN', 'TLAXCALA', 'CHIHUAHUA', 'ZACATECAS',
       'SAN LUIS POTOSÍ', 'CHIAPAS', 'QUERÉTARO', 'CAMPECHE', 'COLIMA'],
      dtype=object)

In [24]:
output.Origin.unique()

array(['Local', 'Alemania', 'Cuba', 'Grecia', 'Nicaragua',
       'Estados Unidos de América', 'Estados Unidos de Am�rica', 'Otro',
       'España', 'Espa�a', 'Colombia', 'Egipto', 'El Salvador', 'Ecuador',
       'Venezuela', 'Haití', 'Hait�', 'República de Honduras',
       'Rep�blica de Honduras', 'Guatemala', 'Francia', 'Bolivia',
       'Dinamarca', 'Japón', 'Jap�n', 'China', 'Perú', 'Per�', 'Camerún',
       'Camer�n', 'Chile', 'Eritrea', 'Canadá', 'Canad�', 'Brasil',
       'Belice', 'Irlanda', 'Costa de Marfil', 'Italia'], dtype=object)

In [25]:
output

,Date_Confirmed,Date_Symptoms,Date_Death,Region,Region_ID,Region_Key,Origin,Status
0,2020-01-06,2020-01-02,9999-99-99,JALISCO,14,JC,Local,2
10281,2020-01-06,2020-01-02,9999-99-99,QUINTANA ROO,23,QR,Local,2
20388,2020-01-06,2020-01-03,9999-99-99,CIUDAD DE MÉXICO,9,DF,Local,2
13310,2020-01-06,2020-01-03,9999-99-99,COAHUILA,5,CL,Local,2
1,2020-01-06,2020-01-03,9999-99-99,JALISCO,14,JC,Local,2
...,...,...,...,...,...,...,...,...
119203,2020-05-05,2020-05-05,9999-99-99,TAMAULIPAS,28,TS,Local,3
160107,2020-05-05,2020-05-05,9999-99-99,TLAXCALA,29,TL,Local,3
160108,2020-05-05,2020-05-05,9999-99-99,TLAXCALA,29,TL,Local,3
66086,2020-05-05,2020-05-05,9999-99-99,VERACRUZ,30,VZ,Local,3


### By status

In [26]:
# Active cases

In [27]:
tbl_1 = output.copy()
tbl_1 = tbl_1[tbl_1.Date_Death.str.startswith(('9999'))] # active cases in patients not deceased

In [28]:
positive = tbl_1[tbl_1['Status']==1]
positive = positive.drop(['Date_Death'], axis=1)

In [29]:
negative = tbl_1[tbl_1['Status']==2]
negative = negative.drop(['Date_Death'], axis=1)

In [30]:
suspects = tbl_1[tbl_1['Status']==3]
suspects = suspects.drop(['Date_Death'], axis=1)

In [31]:
# Deceased patients

In [33]:
tbl_2 = tbl[~tbl.Date_Death.str.startswith(('9999'))] # to-do: check if deceased can have pending testing results in the db
deceased = tbl_2

In [34]:
len(deceased)

4775

In [35]:
# Save

In [36]:
positive.to_csv('../total_positive.csv', index=False)

In [37]:
negative.to_csv('../total_negative.csv', index=False)

In [38]:
suspects.to_csv('../total_suspects.csv', index=False)

In [39]:
deceased.to_csv('../total_deceased.csv', index=False)

In [40]:
# Periodicity on reports

In [78]:
test1 = positive.reset_index()
test1.Date_Confirmed = test1.Date_Confirmed.astype('str')

In [79]:
test1 = test1[test1.Date_Confirmed.str.startswith(('2020-05-04'))]

In [80]:
len(test1)

82

In [81]:
test2 = suspects.reset_index()
test2.Date_Confirmed = test2.Date_Confirmed.astype('str')

In [82]:
test2 = test2[test2.Date_Confirmed.str.startswith(('2020-05-04'))]

In [83]:
len(test2)

4875

In [84]:
# Latest time series

In [85]:
latest = output.copy()

In [86]:
latest = latest[latest.Date_Death.str.startswith(('9999'))] # active cases in patients not deceased
latest = latest[latest['Status']==1]
latest = latest[['Date_Confirmed','Date_Symptoms','Region','Region_ID','Region_Key','Origin','Status']]

In [87]:
latest.to_csv('../latest.csv', index=False)